In [2]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results/execute_descentes.csv')


In [158]:
df.head()

,Instance,Algorithme,Seed,Score,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,best_improver_swap_descente,50,1723,4.420
1,instances/atsp_rand_50_75.txt,best_improver_swap_descente,51,1769,4.352
2,instances/atsp_rand_50_75.txt,best_improver_swap_descente,52,1768,4.562
3,instances/atsp_rand_50_75.txt,best_improver_swap_descente,53,1743,4.783
4,instances/atsp_rand_50_75.txt,best_improver_swap_descente,54,1744,4.816


In [108]:
neighbor_relations = ["swap","2opt"]
choice_mehod = ['first', 'best', 'worst']
versus = [('first', 'best'), ('first', 'worst'),('best', 'worst')]
instances = df['Instance'].unique()

In [94]:
def get_score(instance,neighbor_relation,choice_mehod,seed):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"{choice_mehod}_improver_{neighbor_relation}_descente") &
    (df['Seed'] == seed)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,choice_mehod,seed)
    

In [104]:
def gagne(instance,neighbor_relation,seed, vs):
    #print("vs:",vs)
    s1 = get_score(instance,neighbor_relation,vs[0],seed)
    s2 = get_score(instance,neighbor_relation,vs[1],seed)
    return s1 < s2

In [76]:
get_score("instances/atsp_rand_50_75.txt","2opt","first",55)

1940

In [80]:
gagne("instances/atsp_rand_50_75.txt","2opt",50, ("best","first"))

True

In [155]:
result_tuple={"instances" : instances}
for instance in instances:
    #print(instance)
    for nr in neighbor_relations:
        #print(nr)
        for vs in versus:
            #print(vs," : ",end="")
            r=[0,0]
            for seed in df["Seed"].unique():
                if gagne(instance,nr,seed, vs):
                    r[0]+=1
                else:
                    r[1]+=1
            try:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"].append((r[0],r[1]))
            except:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"] = [(r[0],r[1])]
            #print(r)


result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_first_vs_best,swap_first_vs_worst,swap_best_vs_worst,2opt_first_vs_best,2opt_first_vs_worst,2opt_best_vs_worst
0,instances/atsp_rand_50_75.txt,"(6, 4)","(5, 5)","(4, 6)","(7, 3)","(8, 2)","(6, 4)"
1,instances/atsp_rand_50_100.txt,"(5, 5)","(6, 4)","(5, 5)","(9, 1)","(10, 0)","(7, 3)"
2,instances/atsp_rand_60_50.txt,"(4, 6)","(3, 7)","(5, 5)","(8, 2)","(10, 0)","(7, 3)"
3,instances/atsp_rand_60_100.txt,"(4, 6)","(5, 5)","(5, 5)","(9, 1)","(10, 0)","(7, 3)"
4,instances/atsp_rand_70_55.txt,"(7, 3)","(4, 6)","(5, 5)","(10, 0)","(8, 2)","(3, 7)"
5,instances/atsp_rand_70_100.txt,"(5, 5)","(6, 4)","(4, 6)","(9, 1)","(10, 0)","(5, 5)"
6,instances/atsp_rand_80_70.txt,"(3, 7)","(5, 5)","(3, 7)","(10, 0)","(10, 0)","(5, 5)"
7,instances/atsp_rand_80_100.txt,"(6, 4)","(5, 5)","(5, 5)","(9, 1)","(10, 0)","(4, 6)"


In [156]:
result={"instances" : instances}
for instance in instances:
    #print(instance)
    for nr in neighbor_relations:
        #print(nr)
        for vs in versus:
            #print(vs," : ",end="")
            r=[0,0]
            for seed in df["Seed"].unique():
                if gagne(instance,nr,seed, vs):
                    r[0]+=1
                else:
                    r[1]+=1
            try:
                result[f"{nr}_{vs[0]}_vs_{vs[1]}"].append((r[0]))
            except:
                result[f"{nr}_{vs[0]}_vs_{vs[1]}"] = [(r[0])]
            #print(r)


result = pd.DataFrame(result)
result

,instances,swap_first_vs_best,swap_first_vs_worst,swap_best_vs_worst,2opt_first_vs_best,2opt_first_vs_worst,2opt_best_vs_worst
0,instances/atsp_rand_50_75.txt,6,5,4,7,8,6
1,instances/atsp_rand_50_100.txt,5,6,5,9,10,7
2,instances/atsp_rand_60_50.txt,4,3,5,8,10,7
3,instances/atsp_rand_60_100.txt,4,5,5,9,10,7
4,instances/atsp_rand_70_55.txt,7,4,5,10,8,3
5,instances/atsp_rand_70_100.txt,5,6,4,9,10,5
6,instances/atsp_rand_80_70.txt,3,5,3,10,10,5
7,instances/atsp_rand_80_100.txt,6,5,5,9,10,4


In [157]:
mean_swap_first = (result["swap_first_vs_best"].sum() + result["swap_first_vs_worst"].sum())/16
mean_swap_first


4.9375

In [143]:
mean_swap_best = (80-result["swap_first_vs_best"].sum() + result["swap_best_vs_worst"].sum())/16
mean_swap_best 

4.75

In [144]:
mean_swap_worst = (80-result["swap_first_vs_worst"].sum() + 80 - result["swap_best_vs_worst"].sum())/16
mean_swap_worst

5.3125

In [145]:
mean_2opt_first = (result["2opt_first_vs_best"].sum() + result["2opt_first_vs_worst"].sum())/16
mean_2opt_first

9.1875

In [146]:
mean_2opt_best = (80-result["2opt_first_vs_best"].sum() + result["2opt_best_vs_worst"].sum())/16
mean_2opt_best 

3.3125

In [147]:
mean_2opt_worst = (80-result["2opt_first_vs_worst"].sum() + 80 - result["2opt_best_vs_worst"].sum())/16
mean_2opt_worst

2.5

Pour swap gagnant: worst

Pourt 2opt gagnant: first


In [152]:
first = result["swap_first_vs_worst"].sum() + result["2opt_first_vs_worst"].sum()
worst = 80*2 - first
print(first,worst)

115 45


In [165]:
first_swap = df[(df['Algorithme'] == f"first_improver_swap_descente") ]
first_swap["Score"].mean()

1205.825

In [167]:
best_swap = df[(df['Algorithme'] == f"best_improver_swap_descente") ]
best_swap["Score"].mean()

1206.7625

In [166]:
first_2opt = df[(df['Algorithme'] == f"first_improver_2opt_descente") ]
first_2opt["Score"].mean()

1596.2125

In [168]:
best_2opt = df[(df['Algorithme'] == f"best_improver_2opt_descente") ]
best_2opt["Score"].mean()

1801.5